<a href="https://colab.research.google.com/github/nibaskumar93n-debug/Morphoinformatics/blob/main/molecular_docking_web_tool_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬 AutoDock Web Tool - Professional Molecular Docking Platform

**A fully automated molecular docking pipeline with web interface**

---

### Features:
- ✅ Automatic protein structure download from PDB
- ✅ Automatic ligand retrieval from PubChem
- ✅ Automated structure preparation
- ✅ AutoDock Vina integration
- ✅ Interactive results visualization
- ✅ Professional web interface

---

**Instructions:**
1. Run all cells in order (Runtime > Run all)
2. Use the web interface that appears at the bottom
3. Input PDB ID and compound identifier
4. Click 'Run Docking' and wait for results
5. Download results from the files panel

---

In [1]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: INSTALLATION & SETUP
# ═══════════════════════════════════════════════════════════════════════════════

print("🔧 Installing dependencies...\n")
print("This may take 2-3 minutes. Please wait...\n")

import sys
import subprocess

# Install required packages silently
packages = [
    'biopython',
    'requests',
    'pubchempy',
    'py3Dmol',
    'ipywidgets'
]

for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

# Install system packages
!apt-get update -qq > /dev/null 2>&1
!apt-get install -qq autodock-vina > /dev/null 2>&1
!apt-get install -qq openbabel > /dev/null 2>&1

# Install MGLTools for preparation
!wget -q https://ccsb.scripps.edu/mgltools/download/491/mgltools_x86_64Linux2_1.5.7.tar.gz
!tar -xzf mgltools_x86_64Linux2_1.5.7.tar.gz > /dev/null 2>&1
!cd mgltools_x86_64Linux2_1.5.7 && ./install.sh -d /content/mgltools > /dev/null 2>&1
!rm mgltools_x86_64Linux2_1.5.7.tar.gz

print("\n✅ Installation complete!")
print("\n📌 Next: Run the remaining cells to launch the application")

🔧 Installing dependencies...

This may take 2-3 minutes. Please wait...


✅ Installation complete!

📌 Next: Run the remaining cells to launch the application


In [2]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: IMPORT LIBRARIES
# ═══════════════════════════════════════════════════════════════════════════════

import os
import sys
import json
import shutil
from pathlib import Path
import subprocess
import warnings
warnings.filterwarnings('ignore')

# Scientific libraries
import requests
import pubchempy as pcp
from Bio.PDB import PDBParser, PDBIO, Select
from Bio import PDB
import py3Dmol
from IPython.display import display, HTML
import ipywidgets as widgets

# Set up paths
MGLTOOLS_PATH = '/content/mgltools'
PYTHONSH = f'{MGLTOOLS_PATH}/bin/pythonsh'
PREPARE_LIGAND = f'{MGLTOOLS_PATH}/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py'
PREPARE_RECEPTOR = f'{MGLTOOLS_PATH}/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py'

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [3]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: CORE FUNCTIONS - PROTEIN HANDLING
# ═══════════════════════════════════════════════════════════════════════════════

class ProteinHandler:
    """Handles protein structure download and preparation"""

    @staticmethod
    def download_protein(pdb_id, output_dir='.'):
        """Download protein structure from RCSB PDB"""
        pdb_id = str(pdb_id).upper().strip()
        url = f'https://files.rcsb.org/download/{pdb_id}.pdb'

        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()

            output_path = os.path.join(output_dir, f'{pdb_id}.pdb')
            with open(output_path, 'w') as f:
                f.write(response.text)

            return output_path, True, f"Protein {pdb_id} downloaded successfully"
        except Exception as e:
            return None, False, f"Error downloading protein: {str(e)}"

    @staticmethod
    def clean_protein(pdb_file, output_file):
        """Remove water molecules and heteroatoms from protein"""
        class ProteinSelect(Select):
            def accept_residue(self, residue):
                return residue.id[0] == ' '

        try:
            parser = PDBParser(QUIET=True)
            structure = parser.get_structure('protein', pdb_file)

            io = PDBIO()
            io.set_structure(structure)
            io.save(output_file, ProteinSelect())

            return True, "Protein cleaned successfully"
        except Exception as e:
            return False, f"Error cleaning protein: {str(e)}"

    @staticmethod
    def prepare_receptor(pdb_file, output_pdbqt):
        """Prepare receptor for docking"""
        try:
            cmd = [
                PYTHONSH,
                PREPARE_RECEPTOR,
                '-r', pdb_file,
                '-o', output_pdbqt,
                '-A', 'hydrogens',
                '-U', 'nphs_lps_waters'
            ]

            result = subprocess.run(cmd, capture_output=True, text=True, timeout=60)

            if os.path.exists(output_pdbqt):
                return True, "Receptor prepared successfully"
            else:
                return False, f"Preparation failed"
        except Exception as e:
            return False, f"Error preparing receptor: {str(e)}"

print("✅ Protein handling functions loaded")

✅ Protein handling functions loaded


In [4]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: CORE FUNCTIONS - LIGAND HANDLING
# ═══════════════════════════════════════════════════════════════════════════════

class LigandHandler:
    """Handles ligand structure download and preparation"""

    @staticmethod
    def download_ligand(compound_id, output_dir='.'):
        """Download ligand from PubChem"""
        try:
            compound_id = str(compound_id).strip()

            if compound_id.isdigit():
                compounds = pcp.get_compounds(compound_id, 'cid')
            else:
                compounds = pcp.get_compounds(compound_id, 'name')

            if not compounds:
                return None, False, f"Compound '{compound_id}' not found in PubChem"

            compound = compounds[0]

            sdf_file = os.path.abspath(os.path.join(output_dir, f'ligand_{compound.cid}.sdf'))
            pcp.download('SDF', sdf_file, compound.cid, overwrite=True)

            return sdf_file, True, f"Ligand downloaded (PubChem CID: {compound.cid})"
        except Exception as e:
            return None, False, f"Error downloading ligand: {str(e)}"

    @staticmethod
    def convert_to_pdb(sdf_file, pdb_file):
        """Convert SDF to PDB format"""
        try:
            sdf_file = os.path.abspath(sdf_file)
            pdb_file = os.path.abspath(pdb_file)

            cmd = ['obabel', sdf_file, '-O', pdb_file, '--gen3d']
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=30, cwd=os.path.dirname(pdb_file))

            if os.path.exists(pdb_file) and os.path.getsize(pdb_file) > 0:
                return True, "Converted to PDB format"
            else:
                return False, f"Conversion failed: {result.stderr}"
        except Exception as e:
            return False, f"Error converting ligand: {str(e)}"

    @staticmethod
    def prepare_ligand(pdb_file, output_pdbqt):
        """Prepare ligand for docking"""
        original_dir = os.getcwd()
        try:
            # Use absolute paths
            pdb_file = os.path.abspath(pdb_file)
            output_pdbqt = os.path.abspath(output_pdbqt)

            # Verify input file exists
            if not os.path.exists(pdb_file):
                return False, f"Input PDB file not found: {pdb_file}"

            # Change to the directory containing the ligand file
            # MGLTools has issues with absolute paths
            ligand_dir = os.path.dirname(pdb_file)
            ligand_basename = os.path.basename(pdb_file)
            os.chdir(ligand_dir)

            cmd = [
                PYTHONSH,
                PREPARE_LIGAND,
                '-l', ligand_basename,
                '-o', os.path.basename(output_pdbqt),
                '-A', 'hydrogens'
            ]

            result = subprocess.run(cmd, capture_output=True, text=True, timeout=60)

            # Change back to original directory
            os.chdir(original_dir)

            # Check if output was created
            if os.path.exists(output_pdbqt) and os.path.getsize(output_pdbqt) > 0:
                return True, "Ligand prepared successfully"
            else:
                error_msg = result.stderr if result.stderr else result.stdout
                return False, f"Preparation failed: {error_msg[:200]}"
        except Exception as e:
            # Make sure to change back to original directory
            try:
                os.chdir(original_dir)
            except:
                pass
            return False, f"Error preparing ligand: {str(e)}"

print("✅ Ligand handling functions loaded")

✅ Ligand handling functions loaded


In [5]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 5: CORE FUNCTIONS - DOCKING ENGINE
# ═══════════════════════════════════════════════════════════════════════════════

class DockingEngine:
    """Handles molecular docking with AutoDock Vina"""

    @staticmethod
    def calculate_center_and_size(receptor_file):
        """Calculate binding box center and size from receptor"""
        try:
            parser = PDBParser(QUIET=True)
            structure = parser.get_structure('protein', receptor_file)

            coords = []
            for model in structure:
                for chain in model:
                    for residue in chain:
                        for atom in residue:
                            coords.append(atom.coord)

            import numpy as np
            coords = np.array(coords)

            center = coords.mean(axis=0)
            min_coords = coords.min(axis=0)
            max_coords = coords.max(axis=0)
            size = (max_coords - min_coords) + 10

            return center, size
        except:
            return [0, 0, 0], [25, 25, 25]

    @staticmethod
    def run_vina(receptor_pdbqt, ligand_pdbqt, output_pdbqt, center=None, size=None):
        """Run AutoDock Vina docking"""
        try:
            if center is None or size is None:
                temp_pdb = receptor_pdbqt.replace('.pdbqt', '_temp.pdb')
                subprocess.run(['obabel', receptor_pdbqt, '-O', temp_pdb],
                             capture_output=True, timeout=30)
                center, size = DockingEngine.calculate_center_and_size(temp_pdb)
                if os.path.exists(temp_pdb):
                    os.remove(temp_pdb)

            config_content = f"""receptor = {receptor_pdbqt}
ligand = {ligand_pdbqt}
out = {output_pdbqt}

center_x = {center[0]:.3f}
center_y = {center[1]:.3f}
center_z = {center[2]:.3f}

size_x = {size[0]:.3f}
size_y = {size[1]:.3f}
size_z = {size[2]:.3f}

exhaustiveness = 8
num_modes = 9
"""

            config_file = 'vina_config.txt'
            with open(config_file, 'w') as f:
                f.write(config_content)

            cmd = ['vina', '--config', config_file]
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)

            if os.path.exists(output_pdbqt):
                scores = DockingEngine.parse_vina_output(result.stdout)
                return True, scores, "Docking completed successfully"
            else:
                return False, [], "Docking failed"
        except Exception as e:
            return False, [], f"Error during docking: {str(e)}"

    @staticmethod
    def parse_vina_output(output_text):
        """Parse binding affinities from Vina output"""
        scores = []
        for line in output_text.split('\n'):
            parts = line.split()
            if len(parts) >= 2 and parts[0].isdigit():
                try:
                    scores.append(float(parts[1]))
                except:
                    pass
        return scores

print("✅ Docking engine loaded")

✅ Docking engine loaded


In [6]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 6: VISUALIZATION FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════════

class Visualizer:
    """Handles 3D visualization of docking results"""

    @staticmethod
    def visualize_docking(receptor_file, ligand_file, width=800, height=600):
        """Create 3D visualization of docked complex"""
        try:
            with open(receptor_file, 'r') as f:
                receptor_data = f.read()

            if ligand_file.endswith('.pdbqt'):
                ligand_pdb = ligand_file.replace('.pdbqt', '_vis.pdb')
                subprocess.run(['obabel', ligand_file, '-O', ligand_pdb],
                             capture_output=True, timeout=30)
                with open(ligand_pdb, 'r') as f:
                    ligand_data = f.read()
            else:
                with open(ligand_file, 'r') as f:
                    ligand_data = f.read()

            view = py3Dmol.view(width=width, height=height)
            view.addModel(receptor_data, 'pdb')
            view.setStyle({'model': 0}, {'cartoon': {'color': 'spectrum'}})
            view.addModel(ligand_data, 'pdb')
            view.setStyle({'model': 1}, {'stick': {'colorscheme': 'greenCarbon'}})
            view.zoomTo()

            return view
        except Exception as e:
            print(f"Visualization error: {str(e)}")
            return None

print("✅ Visualization functions loaded")

✅ Visualization functions loaded


In [7]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 7: WEB INTERFACE - MAIN APPLICATION
# ═══════════════════════════════════════════════════════════════════════════════

class MolecularDockingApp:
    """Main application with web interface"""

    def __init__(self):
        self.work_dir = '/content/docking_results'
        os.makedirs(self.work_dir, exist_ok=True)

        self.protein_handler = ProteinHandler()
        self.ligand_handler = LigandHandler()
        self.docking_engine = DockingEngine()
        self.visualizer = Visualizer()

        self.setup_ui()

    def setup_ui(self):
        """Create web interface"""

        header = widgets.HTML(
            value="""<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                        padding: 30px; border-radius: 10px; color: white;
                        text-align: center; margin-bottom: 20px;'>
                <h1 style='margin: 0; font-size: 2.5em;'>🧬 AutoDock Pro</h1>
                <p style='margin: 10px 0 0 0; font-size: 1.2em;'>Professional Molecular Docking Platform</p>
            </div>"""
        )

        self.pdb_input = widgets.Text(
            description='PDB ID:',
            placeholder='e.g., 1HSG, 6LU7',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )

        self.compound_input = widgets.Text(
            description='Compound:',
            placeholder='Name or PubChem CID',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )

        self.dock_button = widgets.Button(
            description='🚀 Run Docking',
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px')
        )
        self.dock_button.on_click(self.run_docking)

        self.example_button = widgets.Button(
            description='📋 Load Example',
            button_style='info',
            layout=widgets.Layout(width='200px', height='40px')
        )
        self.example_button.on_click(self.load_example)

        self.progress = widgets.HTML(value='')
        self.output = widgets.Output()

        input_box = widgets.VBox([self.pdb_input, self.compound_input],
                                 layout=widgets.Layout(padding='10px'))

        button_box = widgets.HBox([self.dock_button, self.example_button],
                                  layout=widgets.Layout(padding='10px', justify_content='center'))

        self.ui = widgets.VBox([header, input_box, button_box, self.progress, self.output])

    def load_example(self, b):
        self.pdb_input.value = '1HSG'
        self.compound_input.value = 'aspirin'
        self.update_progress('✅ Example loaded: HIV-1 Protease with Aspirin', 'info')

    def update_progress(self, message, status='info'):
        colors = {'info': '#3498db', 'success': '#2ecc71', 'warning': '#f39c12', 'error': '#e74c3c'}
        self.progress.value = f"""<div style='background-color: {colors[status]}; color: white;
                    padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <strong>{message}</strong></div>"""

    def run_docking(self, b):
        self.output.clear_output()

        with self.output:
            pdb_id = self.pdb_input.value.strip()
            compound_id = self.compound_input.value.strip()

            if not pdb_id or not compound_id:
                self.update_progress('❌ Please enter both PDB ID and Compound ID', 'error')
                return

            print("="*80)
            print("🧬 MOLECULAR DOCKING PIPELINE")
            print("="*80)
            print(f"Protein: {pdb_id}")
            print(f"Ligand: {compound_id}")
            print("="*80)
            print()

            # Download protein
            self.update_progress('📥 Downloading protein structure...', 'info')
            protein_file, success, msg = self.protein_handler.download_protein(pdb_id, self.work_dir)
            print(f"✓ {msg}")
            if not success:
                self.update_progress(f'❌ {msg}', 'error')
                return

            # Clean protein
            self.update_progress('🧹 Cleaning protein structure...', 'info')
            clean_protein = os.path.join(self.work_dir, f'{pdb_id}_clean.pdb')
            success, msg = self.protein_handler.clean_protein(protein_file, clean_protein)
            print(f"✓ {msg}")

            # Prepare receptor
            self.update_progress('⚙️ Preparing receptor...', 'info')
            receptor_pdbqt = os.path.join(self.work_dir, f'{pdb_id}_receptor.pdbqt')
            success, msg = self.protein_handler.prepare_receptor(clean_protein, receptor_pdbqt)
            print(f"✓ {msg}")
            if not success:
                self.update_progress(f'❌ {msg}', 'error')
                return

            # Download ligand
            self.update_progress('📥 Downloading ligand...', 'info')
            ligand_file, success, msg = self.ligand_handler.download_ligand(compound_id, self.work_dir)
            print(f"✓ {msg}")
            if not success:
                self.update_progress(f'❌ {msg}', 'error')
                return

            # Convert ligand
            self.update_progress('🔄 Converting ligand format...', 'info')
            ligand_pdb = os.path.join(self.work_dir, 'ligand.pdb')
            success, msg = self.ligand_handler.convert_to_pdb(ligand_file, ligand_pdb)
            print(f"✓ {msg}")

            # Prepare ligand
            self.update_progress('⚙️ Preparing ligand...', 'info')
            ligand_pdbqt = os.path.join(self.work_dir, 'ligand.pdbqt')
            success, msg = self.ligand_handler.prepare_ligand(ligand_pdb, ligand_pdbqt)
            print(f"✓ {msg}")
            if not success:
                self.update_progress(f'❌ {msg}', 'error')
                return

            # Run docking
            self.update_progress('🎯 Running molecular docking (this may take a few minutes)...', 'info')
            output_pdbqt = os.path.join(self.work_dir, 'docked.pdbqt')
            success, scores, msg = self.docking_engine.run_vina(receptor_pdbqt, ligand_pdbqt, output_pdbqt)
            print(f"✓ {msg}")

            if not success:
                self.update_progress(f'❌ {msg}', 'error')
                return

            # Display results
            print()
            print("="*80)
            print("📊 DOCKING RESULTS")
            print("="*80)

            if scores:
                print(f"\n🏆 Best Binding Affinity: {scores[0]:.2f} kcal/mol\n")
                print("Top 5 Poses:")
                for i, score in enumerate(scores[:5], 1):
                    print(f"  Pose {i}: {score:.2f} kcal/mol")

            print("\n" + "="*80)
            self.update_progress('✅ Docking completed successfully!', 'success')

            # Visualize
            print("\n🔬 Generating 3D visualization...\n")
            view = self.visualizer.visualize_docking(clean_protein, output_pdbqt)
            if view:
                display(view)

            print("\n📦 Results saved in:", self.work_dir)
            print(f"\nFiles available for download:")
            print(f"  • Prepared receptor: {os.path.basename(receptor_pdbqt)}")
            print(f"  • Prepared ligand: {os.path.basename(ligand_pdbqt)}")
            print(f"  • Docked poses: {os.path.basename(output_pdbqt)}")

    def show(self):
        display(self.ui)

print("✅ Application loaded")

✅ Application loaded


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 8: LAUNCH APPLICATION
# ═══════════════════════════════════════════════════════════════════════════════

print("🚀 Launching Molecular Docking Application...\n")
app = MolecularDockingApp()
app.show()

---

## 📚 Quick Reference

### Example PDB IDs:
- **1HSG** - HIV-1 Protease (good for testing)
- **6LU7** - COVID-19 Main Protease
- **4GQQ** - Glucokinase
- **1IEP** - Streptavidin
- **3HTB** - Beta-lactamase

### Example Compounds:
- **aspirin** - Common drug
- **2244** - PubChem CID for Aspirin
- **remdesivir** - Antiviral drug
- **ibuprofen** - Anti-inflammatory
- **caffeine** - Stimulant

### Understanding Results:
- **Binding Affinity**: Measured in kcal/mol
- **More negative = Stronger binding**
- Values < -6 kcal/mol typically indicate good binding
- Values < -8 kcal/mol suggest strong interaction

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# OPTIONAL: Download Results as ZIP
# ═══════════════════════════════════════════════════════════════════════════════

import shutil

result_dir = '/content/docking_results'

if os.path.exists(result_dir) and os.listdir(result_dir):
    zip_file = '/content/docking_results'
    shutil.make_archive(zip_file, 'zip', result_dir)
    print("✅ Results packaged as: docking_results.zip")
    print("📥 Download from: Files panel (left sidebar) → docking_results.zip")
    print(f"\n📊 Total files: {len(os.listdir(result_dir))}")
else:
    print("⚠️ No results yet. Run docking first!")